In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    for dim in ['band', 'g']: 
        if dim in ds: ds = ds.sum(dim)

    for dim in ['igg', 'g', 'i']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim) 
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    for dim in ['band', 'g']:
        if dim in ds: ds = ds.sum(dim)
        
    for dim in ['i', 'igg', 'g']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=10), ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

if 'ng_refs' in PARAM_LBLNEW: 
    doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))
else:
    doc = client.lblnew.overlap_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_flux']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolr']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wflux']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolr']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[9],7
commitnumber,1013f91,5014a19
dv,-,0.001
molecule,"{'ch4': 1.8e-06, 'h2o': 'atmpro', 'n2o': 3.2e-07}","{'h2o': 'atmpro', 'n2o': 3.2e-07, 'ch4': 1.8e-06}"
nv,-,1000
tsfc,300,300


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.120502                    1 -0.058620
0.000750        2 -0.119356                    2 -0.057905
0.001052        3 -0.116891                    3 -0.056297
0.001476        4 -0.114367                    4 -0.054705
0.002070        5 -0.111823                    5 -0.053040
0.002904        6 -0.109294                    6 -0.051394
0.004074        7 -0.106842                    7 -0.049661
0.005714        8 -0.104449                    8 -0.047915
0.008015        9 -0.102183                    9 -0.046173
0.011243       10 -0.100100                   10 -0.044376
0.015771       11 -0.098301                   11 -0.042608
0.022122       12 -0.096895                   12 -0.040784
0.031031       13 -0.096065                   13 -0.039030
0.043528       14 -0.095186                   14 -0.036687
0.061057       15 -0.084306                   15 -0.027392
0.085645       16 -0.055524                   16 -0.005931
0.120136       17 -0.014390                   17  0.024779
0.168516       18  0.035145                   18  0.062622
0.236378       19  0.094252                   19  0.108736
0.331549       20  0.164610                   20  0.164690
0.465100       21  0.248696                   21  0.232159
0.652400       22  0.337965                   22  0.304013
0.915100       23  0.356815                   23  0.328160
1.283650       24  0.300332                   24  0.296733
1.800600       25  0.228455                   25  0.246418
2.525700       26  0.154988                   26  0.185678
3.542800       27  0.100046                   27  0.130549
4.969550       28  0.064671                   28  0.085974
6.970850       29  0.043115                   29  0.052831
9.778100       30  0.030150                   30  0.030635
13.715850      31  0.021765                   31  0.017745
19.239350      32  0.015375                   32  0.010585
26.987250      33  0.009643                   33  0.005494
37.855300      34  0.003901                   34 -0.000448
53.100050      35 -0.006345                   35 -0.012088
73.887500      36 -0.020203                   36 -0.028188
97.662500      37 -0.031638                   37 -0.041345
121.437500     38 -0.035660                   38 -0.046162
145.212500     39 -0.033615                   39 -0.044582
168.987500     40 -0.032046                   40 -0.042529
192.762500     41 -0.029954                   41 -0.039659
216.537500     42 -0.026726                   42 -0.035609
240.312500     43 -0.022003                   43 -0.030308
264.087500     44 -0.015741                   44 -0.023875
287.862500     45 -0.007946                   45 -0.015079
311.637500     46  0.001323                   46 -0.004952
335.412500     47  0.011842                   47  0.004650
359.187500     48  0.022895                   48  0.015102
382.962500     49  0.033000                   49  0.026264
406.737500     50  0.044397                   50  0.038508
430.512500     51  0.056614                   51  0.050298
454.287500     52  0.069315                   52  0.061787
478.062500     53  0.082177                   53  0.075037
501.837500     54  0.094785                   54  0.089249
525.612500     55  0.107009                   55  0.105021
549.387500     56  0.118478                   56  0.118732
573.162500     57  0.129157                   57  0.130832
596.937500     58  0.139232                   58  0.140577
620.712500     59  0.148945                   59  0.149694
644.487500     60  0.159106                   60  0.160183
668.262500     61  0.169942                   61  0.172854
692.037500     62  0.179510                   62  0.185652
715.812500     63  0.183190                   63  0.192733
739.587500     64  0.184146                   64  0.198983
763.362500     65  0.170433                   65  0.188566
78

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -13.104923   0.000000 -13.104923 -13.078203  9.028963e-08   
0.000624    2     -13.104936   0.000004 -13.104932 -13.078210  2.464031e-06   
0.000876    3     -13.104941   0.000005 -13.104936 -13.078212  3.432673e-06   
0.001229    4     -13.104948   0.000007 -13.104941 -13.078216  4.825732e-06   
0.001723    5     -13.104958   0.000011 -13.104947 -13.078221  6.827477e-06   
0.002417    6     -13.104972   0.000015 -13.104957 -13.078229  9.705018e-06   
0.003391    7     -13.104991   0.000022 -13.104969 -13.078239  1.383892e-05   
0.004757    8     -13.105017   0.000031 -13.104986 -13.078253  1.978217e-05   
0.006672    9     -13.105054   0.000044 -13.105010 -13.078272  2.832327e-05   
0.009359    10    -13.105106   0.000063 -13.105043 -13.078299  4.059178e-05   
0.013128    11    -13.105178   0.000091 -13.105087 -13.078337  5.821907e-05   
0.018415    12    -13.105279   0.000130 -13.105149 -13.078389  8.352514e-05   
0.025830    13    -13.105419   0.000185 -13.105234 -13.078461  1.198667e-04   
0.036232    14    -13.105617   0.000264 -13.105352 -13.078561  1.719973e-04   
0.050823    15    -13.105894   0.000377 -13.105517 -13.078700  2.472891e-04   
0.071291    16    -13.106271   0.000550 -13.105721 -13.078883  3.645958e-04   
0.100000    17    -13.106754   0.000844 -13.105910 -13.079106  5.665711e-04   
0.140271    18    -13.107337   0.001359 -13.105979 -13.079345  9.243575e-04   
0.196760    19    -13.107991   0.002247 -13.105744 -13.079555  1.553660e-03   
0.275997    20    -13.108620   0.003761 -13.104859 -13.079632  2.651260e-03   
0.387100    21    -13.108997   0.006305 -13.102693 -13.079362  4.548890e-03   
0.543100    22    -13.108621   0.010524 -13.098097 -13.078327  7.804144e-03   
0.761700    23    -13.106588   0.017243 -13.089345 -13.075853  1.320349e-02   
1.068500    24    -13.102652   0.026274 -13.076378 -13.071723  2.100053e-02   
1.498800    25    -13.097397   0.036328 -13.061069 -13.066307  3.071170e-02   
2.102400    26    -13.091100   0.046366 -13.044734 -13.059804  4.182992e-02   
2.949000    27    -13.084363   0.055171 -13.029191 -13.052676  5.332451e-02   
4.136600    28    -13.077452   0.062335 -13.015117 -13.045161  6.417688e-02   
5.802500    29    -13.070456   0.068101 -13.002355 -13.037406  7.338938e-02   
8.139200    30    -13.063503   0.073082 -12.990421 -13.029627  8.023577e-02   
11.417000   31    -13.056825   0.078111 -12.978714 -13.022229  8.473344e-02   
16.014700   32    -13.050892   0.084032 -12.966860 -13.015954  8.812372e-02   
22.464000   33    -13.046622   0.091508 -12.955114 -13.012262  9.251960e-02   
31.510500   34    -13.045684   0.100902 -12.944781 -13.013958  1.001029e-01   
44.200100   35    -13.050899   0.111982 -12.938917 -13.026325  1.117977e-01   
62.000000   36    -13.073050   0.120753 -12.952296 -13.064092  1.240742e-01   
85.775000   37    -13.132994   0.123799 -13.009195 -13.151622  1.322102e-01   
109.550000  38    -13.225350   0.127053 -13.098298 -13.274723  1.388584e-01   
133.325000  39    -13.342357   0.143631 -13.198726 -13.421756  1.558713e-01   
157.100000  40    -13.473504   0.180108 -13.293396 -13.581861  1.904065e-01   
180.875000  41    -13.616734   0.233087 -13.383647 -13.752393  2.411503e-01   
204.650000  42    -13.771013   0.303006 -13.468006 -13.931826  3.088809e-01   
228.425000  43    -13.937949   0.394673 -13.543276 -14.121485  3.982445e-01   
252.200000  44    -14.118565   0.513322 -13.605243 -14.322035  5.134297e-01   
275.975000  45    -14.314535   0.664960 -13.649576 -14.534933  6.590823e-01   
299.750000  46    -14.527968   0.856015 -13.671954 -14.762011  8.436885e-01   
323.525000  47    -14.761247   1.093020 -13.668227 -15.005405  1.073135e+00   
347.300000  48    -15.017011   1.382133 -13.634878 -15.268279  1.349106e+00

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')